In [ ]:
import os
import requests
import pandas as pd
import json
import csv

In [ ]:
csv_url = "https://ogd.parl.apps.be.ch/data/abstimmungsliste.csv"
csv_filename = "abstimmungsliste.csv"

if not os.path.exists(csv_filename):
    response = requests.get(csv_url)
    response.raise_for_status()
    with open(csv_filename, "wb") as f:
        f.write(response.content)

In [ ]:
dataframe = pd.read_csv(csv_filename)

display(dataframe.head(5))

In [ ]:
def vote_to_int(vote):
    if vote == "Enthalten":
        return 0
    elif vote == "Ja":
        return 1
    elif vote == "Nein":
        return 2
    else:
        return -1

# Get all unique abstimmung_uid values in the order they appear
abstimmung_uids = list(dict.fromkeys(dataframe["abstimmung_uid"]))

# Prepare header for CSV
header = ["id", "first_name", "last_name"] + abstimmung_uids

# Prepare a mapping from mitglied_uid to their row data
rows = {}
for _, row in dataframe.iterrows():
    mitglied_uid = row["mitglied_uid"]
    if mitglied_uid not in rows:
        rows[mitglied_uid] = {
            "id": mitglied_uid,
            "first_name": row["mitglied_vorname"],
            "last_name": row["mitglied_name"],
            **{uid: -1 for uid in abstimmung_uids},  # fill with -1 by default
        }
    abstimmung_uid = row["abstimmung_uid"]
    stimme = vote_to_int(row["abstimmungsliste_stimme"])
    rows[mitglied_uid][abstimmung_uid] = stimme

# Write to CSV
with open("votes.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    for row in rows.values():
        writer.writerow(row)
print("Result written to votes.csv")
